In [26]:
import os, math, random, numpy as np, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_LEVEL_DIR = "./levels"
OUTPUT_DIR      = "./generated_levels"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# auto‑extract symbols
symbols = {"-"}
for fn in os.listdir(TRAIN_LEVEL_DIR):
    if fn.endswith(".txt"):
        with open(os.path.join(TRAIN_LEVEL_DIR, fn)) as f:
            for line in f:
                symbols.update(line.rstrip())
symbols = sorted(symbols)
int_to_char = {i: ch for i, ch in enumerate(symbols)}
char_to_int = {ch: i for i, ch in int_to_char.items()}

LEVEL_HEIGHT  = 16              
LEVEL_WIDTH   = 200
PATCH         = 16             
LATENT_DIM    = 128
BATCH_SIZE    = 64
EPOCHS        = 30
MAX_TILE      = len(int_to_char) - 1

GROUND = {char_to_int.get(c) for c in ("X","S")} - {None}
ENEMIES= {char_to_int.get(c) for c in ("G","k","g","Y","y")} - {None}
START  = char_to_int.get("M", 1)
GOAL   = char_to_int.get("F", 2)

print("Tile set:", "".join(symbols))

Tile set: #%-12@BCEFGKLMQSTUXbot|


In [27]:
def ascii_to_grid(lines):
    # Convert a list of ASCII strings to a numeric grid using tile mappings
    g = np.zeros((LEVEL_HEIGHT, LEVEL_WIDTH), np.uint8)
    for r, l in enumerate(lines[:LEVEL_HEIGHT]):
        for c, ch in enumerate(l.rstrip()[:LEVEL_WIDTH]):
            g[r, c] = char_to_int.get(ch, 0)  # Default to 0 (usually sky) if unknown
    return g

def iter_patches(grid):
    # Yield non-overlapping 16-tile-wide patches from the full grid
    for x in range(0, LEVEL_WIDTH - PATCH + 1, PATCH):
        yield grid[:, x:x+PATCH]

class Mario16(Dataset):
    def __init__(self, folder):
        buf = []
        # Read and process all .txt level files in the folder
        for fn in os.listdir(folder):
            if fn.endswith(".txt"):
                with open(os.path.join(folder, fn)) as f:
                    buf.extend(iter_patches(ascii_to_grid(f.readlines())))
        self.data = np.stack(buf)  # Stack patches into a numpy array

    def __len__(self):  
        return len(self.data)

    def __getitem__(self, i):
        # Normalize tile IDs and convert to PyTorch tensor with channel dimension
        x = self.data[i] / MAX_TILE
        return torch.tensor(x, dtype=torch.float32).unsqueeze(0)

# Create DataLoader for training batches of 16×16 level segments
loader = DataLoader(Mario16(TRAIN_LEVEL_DIR),
                    batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

print("16×16 patches:", len(loader.dataset))


16×16 patches: 12012


In [28]:
def w_init(m):
    # Initialize weights for Conv and Linear layers using a normal distribution
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.normal_(m.weight, 0, 0.02)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(LATENT_DIM, 1024 * 4 * 4), nn.ReLU(True)  # Map latent vector to 4x4 feature map
        )
        self.up1 = nn.Sequential(  # Upsample: 4×4 → 8×8
            nn.ConvTranspose2d(1024, 512, 4, 2, 1),
            nn.BatchNorm2d(512), nn.ReLU(True)
        )
        self.up2 = nn.Sequential(  # Upsample: 8×8 → 16×16 + refinement
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 128, 3, 1, 1),
            nn.BatchNorm2d(128), nn.ReLU(True)
        )
        self.out = nn.Sequential(  # Final output layer with Sigmoid to normalize to [0, 1]
            nn.Conv2d(128, 1, 3, 1, 1), nn.Sigmoid()
        )

    def forward(self, z):
        x = self.fc(z).view(-1, 1024, 4, 4)         # Reshape to 4×4 feature map
        x = self.up1(x)                             # Upsample to 8×8
        x = x + torch.randn_like(x) * 0.05          # Add noise for diversity
        x = self.up2(x)                             # Upsample to 16×16
        return self.out(x)                          # Output shape: (B,1,16,16)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.body = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1), nn.LeakyReLU(0.2, True),     # Downsample: 16×16 → 8×8
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True)                                 # Downsample: 8×8 → 4×4
        )
        self.avg = nn.AdaptiveAvgPool2d((4, 4))                     # Ensure consistent 4×4 output
        self.head = nn.Sequential(                                  # Final binary classification head
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.head(self.avg(self.body(x)))  # Output: probability of being real

# Instantiate generator and discriminator and apply weight initialization
Gnet, Dnet = Generator().to(device), Discriminator().to(device)
Gnet.apply(w_init)
Dnet.apply(w_init)

Discriminator(
  (body): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (avg): AdaptiveAvgPool2d(output_size=(4, 4))
  (head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=1, bias=True)
    (2): Sigmoid()
  )
)

In [29]:
from dataclasses import dataclass

@dataclass
class DifficultyConfig:
    # Stores parameters that control level difficulty
    pit_freq: float         # Expected number of pits per 100 tiles
    pit_width_mean: float   # Average pit width in tiles
    sky_max_clear: float    # Probability of removing bad sky tiles in top row
    sky_min_clear: float    # Probability of removing bad sky tiles near ground
    sky_curve: str = "linear"  # Tapering curve for sky clearing: 'exp', 'linear', or 'quad'
    seed: int = 0              # Random seed for reproducible decorations (0 = random)

# Predefined difficulty settings (can be modified as needed)
PRESETS = {
    "easy":   DifficultyConfig(0.05, 1.0, 0.98, 0.75),
    "medium":   DifficultyConfig(0.10, 1.5, 0.95, 0.65),
    "hard": DifficultyConfig(0.30, 2.0, 0.90, 0.50),
}


In [30]:
import torch, torch.nn as nn
import random
import numpy as np

to_unit = lambda t: t  # Generator output is already in [0, 1]

GROUND_TILES = {char_to_int[c] for c in {'X', 'S'} if c in char_to_int}
AIR_ID = char_to_int['-']
COIN_ID = char_to_int.get('o', AIR_ID)
MAX_TILE = max(char_to_int.values())
SOLID_IDS = set(range(MAX_TILE + 1)) - {AIR_ID, COIN_ID}

def soft_is(u, ids, temp: float = 8.0):
    # Smooth mask for selected tile IDs using sigmoids
    if not ids:
        return torch.zeros_like(u)
    masks = []
    M = MAX_TILE + 1
    for tid in ids:
        lo, hi = tid / M, (tid + 1) / M
        masks.append(torch.sigmoid((u - lo) * temp) * torch.sigmoid((hi - u) * temp))
    return torch.stack(masks).sum(0).clamp(0, 1)

def debug_tile_distributions(u):
    # Print tile frequencies for debugging
    tiles = torch.round(u * MAX_TILE).long()
    unique_tiles, counts = torch.unique(tiles, return_counts=True)
    total = torch.numel(tiles)
    for i, t in enumerate(unique_tiles.cpu().numpy()):
        print(f"Tile {t} ({int_to_char.get(t, '?')}): {counts[i].item()/total*100:.2f}%")

def ground_excess(u, target: float = 0.6):
    # Penalize too much or too little ground coverage in bottom two rows
    tiles = torch.round(u * MAX_TILE).long()
    ground_mask = torch.zeros_like(tiles, dtype=torch.float32)
    for tid in GROUND_TILES:
        ground_mask += (tiles == tid).float()
    ground_mask = ground_mask.clamp(0, 1)
    bottom_rows = ground_mask[:, -2:, :]
    coverage = bottom_rows.mean((1, 2))
    deviation = (coverage - target) ** 2
    low_penalty = torch.where(coverage < 0.2, (0.2 - coverage) * 2.0, torch.zeros_like(coverage))
    return (deviation + low_penalty).mean() * 0.5

def stretch_pen(u, span: int = 6, gap_target: float = 0.3):
    # Penalize long solid sections with no gaps in the ground
    tiles = torch.round(u * MAX_TILE).long()
    ground_mask = torch.zeros_like(tiles, dtype=torch.float32)
    for tid in GROUND_TILES:
        ground_mask += (tiles == tid).float()
    ground_mask = ground_mask.clamp(0, 1)
    bottom_rows = ground_mask[:, -2:, :]
    bottom_row_avg = bottom_rows.mean(1)
    if bottom_row_avg.shape[1] < span:
        return torch.tensor(0.0, device=u.device)
    windows = bottom_row_avg.unfold(1, span, 1)
    ground_ratio = windows.mean(-1)
    gap_ratio = 1.0 - ground_ratio
    insufficient_gaps = torch.clamp(gap_target - gap_ratio, min=0) ** 2
    penalties = insufficient_gaps.sum(1) / windows.shape[1]
    noise = torch.rand_like(penalties) * 0.05
    return (penalties + noise).mean() * 4.0

def sky_solid_pen(u, top_rows: int = 8):
    # Penalize solid blocks appearing in the top of the level
    tiles = torch.round(u * MAX_TILE).long()
    sky_tiles = {AIR_ID, COIN_ID}
    sky_mask = torch.ones_like(tiles, dtype=torch.float32)
    for tid in sky_tiles:
        sky_mask *= (tiles != tid).float()
    actual_top = min(top_rows, tiles.shape[1])
    top_section = sky_mask[:, :actual_top, :]
    row_indices = torch.arange(actual_top, device=u.device)
    row_weights = torch.exp(-0.3 * row_indices)
    weighted = top_section * row_weights.view(1, -1, 1)
    total_possible = row_weights.sum() * tiles.shape[2]
    solid_ratio = weighted.sum((1, 2)) / total_possible
    noise = torch.rand_like(solid_ratio) * 0.05
    return (solid_ratio ** 2 + noise).mean() * 5.0

def check_loss_balance(ground_loss, stretch_loss, sky_loss):
    # Print loss composition for debugging
    total = ground_loss + stretch_loss + sky_loss
    if total > 0:
        print(f"Loss balance check: ground={ground_loss/total:.2f}, stretch={stretch_loss/total:.2f}, sky={sky_loss/total:.2f}")
    else:
        print("Total structure loss is zero!")

bce = nn.BCELoss()
optG = torch.optim.Adam(Gnet.parameters(), 1e-4, betas=(0.5, 0.999))
optD = torch.optim.Adam(Dnet.parameters(), 1e-4, betas=(0.5, 0.999))
λ_struct = 1.0  # Strength of structure loss

g_losses, d_losses, struct_losses = [], [], []
ground_losses, stretch_losses, sky_losses = [], [], []

debug_mode = True
sample_interval = 5
torch.manual_seed(42)
random.seed(42)
max_grad_norm = 1.0  # Clip gradients for stability

print("Starting training with:")
print(f"  GROUND_TILES: {GROUND_TILES}")
print(f"  AIR_ID: {AIR_ID}")
print(f"  COIN_ID: {COIN_ID}")
print(f"  SOLID_IDS: {len(SOLID_IDS)} tiles")

for ep in range(EPOCHS):
    tot_g, tot_d, tot_struct, n_batches = 0.0, 0.0, 0.0, 0
    tot_ground, tot_stretch, tot_sky = 0.0, 0.0, 0.0
    debug_this_epoch = debug_mode and (ep % sample_interval == 0)
    sky_weight_factor = max(1.0, 3.0 - ep * 0.1)  # Decrease sky penalty over time

    for real in loader:
        real = real.to(device)
        batch_size = real.size(0)

        optD.zero_grad()
        real_pred = Dnet(real)
        d_real_loss = bce(real_pred, torch.ones_like(real_pred))
        z = torch.randn(batch_size, LATENT_DIM, device=device)
        fake = Gnet(z).detach()
        d_fake_loss = bce(Dnet(fake), torch.zeros_like(real_pred))
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        torch.nn.utils.clip_grad_norm_(Dnet.parameters(), max_grad_norm)
        optD.step()

        optG.zero_grad()
        z = torch.randn(batch_size, LATENT_DIM, device=device)
        gen = Gnet(z)
        gen_pred = Dnet(gen)
        g_adv_loss = bce(gen_pred, torch.ones_like(gen_pred))

        unit = to_unit(gen)
        ground_ex = ground_excess(unit, target=0.6)
        stretch = stretch_pen(unit, span=6, gap_target=0.3)
        sky_pen = sky_solid_pen(unit, top_rows=8)

        if debug_this_epoch and n_batches == 0:
            check_loss_balance(ground_ex.item(), stretch.item(), sky_pen.item())

        ground_weight = 1.0
        stretch_weight = 1.0
        sky_weight = 1.5 * sky_weight_factor

        weighted_ground = ground_weight * ground_ex
        weighted_stretch = stretch_weight * stretch
        weighted_sky = sky_weight * sky_pen
        struct_loss = weighted_ground + weighted_stretch + weighted_sky

        g_loss = g_adv_loss + λ_struct * struct_loss
        g_loss.backward()
        torch.nn.utils.clip_grad_norm_(Gnet.parameters(), max_grad_norm)
        optG.step()

        tot_g += g_loss.item()
        tot_d += d_loss.item()
        tot_struct += struct_loss.item()
        tot_ground += weighted_ground.item()
        tot_stretch += weighted_stretch.item()
        tot_sky += weighted_sky.item()
        n_batches += 1

    g_losses.append(tot_g / n_batches)
    d_losses.append(tot_d / n_batches)
    struct_losses.append(tot_struct / n_batches)
    ground_losses.append(tot_ground / n_batches)
    stretch_losses.append(tot_stretch / n_batches)
    sky_losses.append(tot_sky / n_batches)

    print(f"ep{ep:02} | D={d_losses[-1]:.3f} | G={g_losses[-1]:.3f} | struct={struct_losses[-1]:.3f}")
    print(f"    Structure components: ground={ground_losses[-1]:.3f}, stretch={stretch_losses[-1]:.3f}, sky={sky_losses[-1]:.3f}")


Starting training with:
  GROUND_TILES: {18, 15}
  AIR_ID: 2
  COIN_ID: 20
  SOLID_IDS: 21 tiles
Loss balance check: ground=0.07, stretch=0.02, sky=0.92
ep00 | D=0.817 | G=18.778 | struct=17.074
    Structure components: ground=0.350, stretch=0.100, sky=16.623
ep01 | D=0.274 | G=15.032 | struct=12.038
    Structure components: ground=0.355, stretch=0.100, sky=11.584
ep02 | D=0.920 | G=6.431 | struct=5.138
    Structure components: ground=0.366, stretch=0.100, sky=4.672
ep03 | D=0.737 | G=5.089 | struct=3.632
    Structure components: ground=0.367, stretch=0.100, sky=3.165
ep04 | D=0.795 | G=4.069 | struct=2.708
    Structure components: ground=0.369, stretch=0.100, sky=2.239
Loss balance check: ground=0.36, stretch=0.10, sky=0.55
ep05 | D=0.874 | G=3.680 | struct=2.421
    Structure components: ground=0.371, stretch=0.100, sky=1.950
ep06 | D=0.903 | G=3.413 | struct=2.199
    Structure components: ground=0.369, stretch=0.100, sky=1.730
ep07 | D=0.927 | G=3.163 | struct=2.008
    Struct

In [31]:
import os, re, glob, math, random, torch

to_int = lambda t: torch.round(t * MAX_TILE).long()  # Convert normalized tensor to integer tile IDs

def _next_name(folder, pat="level_*.txt"):
    # Find the next available level filename in the folder
    ids = [int(m.group(1)) for p in glob.glob(os.path.join(folder, pat))
           if (m := re.search(r"level_(\d+)\.txt$", os.path.basename(p)))]
    return f"level_{(max(ids)+1) if ids else 0:04}.txt"

def clean_chunk(ch):
    # Force bottom two rows to be solid ground
    ch[:, -2:, :] = list(GROUND_TILES)[0]
    return ch

def stitch(parts):
    # Combine horizontal patches into full level shape
    return torch.cat(parts, dim=2)[:, :LEVEL_HEIGHT, :LEVEL_WIDTH]

def carve_random_pits(lvl, cfg: DifficultyConfig):
    # Add random pits based on difficulty configuration
    if cfg.seed:
        random.seed(cfg.seed)
    y0, y1 = LEVEL_HEIGHT - 2, LEVEL_HEIGHT - 1
    x = 2  # Start with 2 solid tiles on the left
    while x < LEVEL_WIDTH - 2:
        if random.random() < cfg.pit_freq:
            width = max(1, int(random.gauss(cfg.pit_width_mean, 0.7)))
            lvl[0, y0:y1 + 1, x:x + width] = AIR_ID
            x += width + 2  # Leave space before next pit
        else:
            x += 1
    return lvl

SKY_ROWS = 6  # Number of rows at the top considered as sky
ALLOWED_IDS_CPU = torch.tensor([AIR_ID, COIN_ID])  # Only air and coins are allowed in sky

def _rowwise_probs(H, max_p, min_p, mode="exp", exp_base=4.0, *, device="cpu"):
    # Compute deletion probability for each sky row (top to bottom)
    t = torch.linspace(0, 1, H, device=device)
    if mode == "linear":
        probs = min_p + (max_p - min_p) * (1 - t)
    elif mode == "quad":
        probs = min_p + (max_p - min_p) * (1 - t) ** 2
    elif mode == "exp":
        probs = min_p + (max_p - min_p) * (exp_base ** (-t))
    else:
        raise ValueError("mode must be 'linear', 'quad', or 'exp'")
    return probs

def prune_sky(lvl, cfg: DifficultyConfig):
    # Remove disallowed tiles from the top sky rows using row-based probabilities
    device_lvl = lvl.device
    H = min(SKY_ROWS, lvl.shape[1])
    probs = _rowwise_probs(H, cfg.sky_max_clear, cfg.sky_min_clear, cfg.sky_curve, device=device_lvl)
    allowed_ids = ALLOWED_IDS_CPU.to(device_lvl)

    for y in range(H):
        p = probs[y].item()
        row = lvl[:, y, :]
        keep_mask = torch.isin(row, allowed_ids)
        delete_mask = (~keep_mask) & (torch.rand_like(row, dtype=torch.float) < p)
        row[delete_mask] = AIR_ID
    return lvl

def ascii_grid(g):
    # Convert level tensor into ASCII strings row by row
    return ["".join(int_to_char[int(t)] for t in g[0, r, :]) for r in range(LEVEL_HEIGHT)]

def generate_level(fname=None, preset="medium"):
    # Generate one level and save it as a .txt file
    cfg = PRESETS[preset] if isinstance(preset, str) else preset
    if cfg.seed:
        random.seed(cfg.seed)

    fname = fname or _next_name(OUTPUT_DIR)
    parts = []
    chunks = math.ceil(LEVEL_WIDTH / PATCH)
    for _ in range(chunks):
        with torch.no_grad():
            z = torch.randn(1, LATENT_DIM, device=device)
            patch = Gnet(z).cpu()
            parts.append(clean_chunk(to_int(patch).squeeze(0)))

    lvl = stitch(parts)
    lvl = carve_random_pits(lvl, cfg)
    lvl = prune_sky(lvl, cfg)

    # Place Mario near the start, 6 tiles above the ground
    start_x = 1
    start_y = LEVEL_HEIGHT - 8
    lvl[0, LEVEL_HEIGHT - 2, 1] = AIR_ID  # Clear default Mario location
    lvl[0, start_y, start_x] = START

    # Place goal near the end on the ground
    lvl[0, LEVEL_HEIGHT - 2, LEVEL_WIDTH - 2] = GOAL

    with open(os.path.join(OUTPUT_DIR, fname), "w") as f:
        f.write("\n".join(ascii_grid(lvl)))
    print("saved", fname)

for preset in ["easy", "medium", "hard"]:
    generate_level(preset=preset)


saved level_0229.txt
saved level_0230.txt
saved level_0231.txt


In [32]:
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from scipy.stats import entropy
import seaborn as sns
from matplotlib.colors import ListedColormap
from collections import Counter

# Create directory to save figures
FIGURES_DIR = os.path.join(OUTPUT_DIR, "figures")
os.makedirs(FIGURES_DIR, exist_ok=True)

def plot_loss_curves():
    # Plot GAN and structure loss over training
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(range(EPOCHS), d_losses, label='Discriminator Loss')
    plt.plot(range(EPOCHS), g_losses, label='Generator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('GAN Training Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(range(EPOCHS), struct_losses, label='Structure Loss', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Structure Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'loss_curves.png'), dpi=300)
    plt.close()

def generate_sample_levels(num_samples=30, seed=None):
    # Generate sample levels from the generator using all presets
    if seed is not None:
        torch.manual_seed(seed)
        random.seed(seed)

    levels = []
    for i in range(num_samples):
        parts = []
        chunks = math.ceil(LEVEL_WIDTH / PATCH)
        for _ in range(chunks):
            with torch.no_grad():
                z = torch.randn(1, LATENT_DIM, device=device)
                parts.append(clean_chunk(to_int(Gnet(z).cpu()).squeeze(0)))
        lvl = stitch(parts)
        cfg = PRESETS[["easy", "medium", "hard"][i % 3]]
        lvl = carve_random_pits(lvl, cfg)
        lvl = prune_sky(lvl, cfg)
        levels.append(lvl)
    return levels

def calculate_metrics(levels):
    # Compute leniency, linearity, entropy for each level
    results = []

    for lvl in levels:
        arr = lvl[0].cpu().numpy()

        enemy_count = np.sum(np.isin(arr, list(ENEMIES)))
        pit_count = 0
        pit_widths = []
        in_pit = False
        current_width = 0

        for x in range(arr.shape[1]):
            is_solid = any(arr[y, x] in GROUND_TILES for y in range(LEVEL_HEIGHT - 2, LEVEL_HEIGHT))
            if not is_solid:
                if not in_pit:
                    in_pit = True
                    pit_count += 1
                current_width += 1
            elif in_pit:
                pit_widths.append(current_width)
                current_width = 0
                in_pit = False
        if in_pit:
            pit_widths.append(current_width)

        avg_pit_width = np.mean(pit_widths) if pit_widths else 0
        leniency = (enemy_count / (LEVEL_HEIGHT * LEVEL_WIDTH)) + (pit_count * avg_pit_width / LEVEL_WIDTH)

        ground_heights = []
        for x in range(arr.shape[1]):
            for y in range(LEVEL_HEIGHT):
                if arr[y, x] in GROUND_TILES:
                    ground_heights.append(y)
                    break
            else:
                ground_heights.append(LEVEL_HEIGHT - 1)

        linearity = np.std(ground_heights)
        tile_counts = Counter(arr.flatten())
        probs = np.array(list(tile_counts.values())) / arr.size
        tile_entropy = entropy(probs)

        results.append({
            'leniency': leniency,
            'linearity': linearity,
            'tile_entropy': tile_entropy,
            'pit_count': pit_count,
            'avg_pit_width': avg_pit_width
        })

    return results

def plot_expressive_range(metrics):
    # Plot scatter of leniency vs. linearity colored by entropy
    plt.figure(figsize=(10, 8))

    leniency = [m['leniency'] for m in metrics]
    linearity = [m['linearity'] for m in metrics]
    tile_entropy = [m['tile_entropy'] for m in metrics]
    norm_entropy = [(e - min(tile_entropy)) / (max(tile_entropy) - min(tile_entropy) + 1e-10) for e in tile_entropy]

    sc = plt.scatter(linearity, leniency, c=norm_entropy, cmap='viridis',
                     alpha=0.7, s=100, edgecolors='w')

    plt.colorbar(sc, label='Normalized Tile Entropy')
    plt.xlabel('Linearity (terrain variation)')
    plt.ylabel('Leniency (lower = easier)')
    plt.title('Expressive Range of Generated Levels')
    plt.grid(True, alpha=0.3)

    for i in [0, 10, 20, 29]:
        if i < len(metrics):
            plt.annotate(f"Level {i}", (linearity[i], leniency[i]),
                         textcoords="offset points", xytext=(0,10), ha='center')

    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'expressive_range.png'), dpi=300)
    plt.close()

def plot_tile_distribution(levels):
    # Show frequency of each tile type and entropy distribution
    plt.figure(figsize=(14, 10))
    tile_type_data = {ch: [] for ch in symbols}

    for lvl in levels:
        arr = lvl[0].cpu().numpy()
        total = arr.size
        for i, ch in int_to_char.items():
            count = np.sum(arr == i)
            tile_type_data[ch].append((count / total) * 100)

    filtered_data = {ch: data for ch, data in tile_type_data.items() if sum(data) > 0 and ch != '-'}

    plt.subplot(2, 1, 1)
    boxplot = plt.boxplot([filtered_data[ch] for ch in filtered_data],
                          labels=list(filtered_data.keys()), patch_artist=True)

    colors = plt.cm.tab10(np.linspace(0, 1, len(filtered_data)))
    for patch, color in zip(boxplot['boxes'], colors):
        patch.set_facecolor(color)

    plt.ylabel('Percentage of Level (%)')
    plt.title('Distribution of Tile Types')
    plt.grid(True, alpha=0.3)

    plt.subplot(2, 1, 2)
    tile_entropies = [m['tile_entropy'] for m in calculate_metrics(levels)]
    sns.histplot(tile_entropies, kde=True)
    plt.xlabel('Tile Entropy')
    plt.ylabel('Frequency')
    plt.title('Tile Entropy Across Levels')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'tile_distribution.png'), dpi=300)
    plt.close()

def plot_tile_heatmaps(levels):
    # Heatmaps showing where tile types appear on average
    combined_heatmap = np.zeros((LEVEL_HEIGHT, LEVEL_WIDTH))
    ground_heatmap = np.zeros((LEVEL_HEIGHT, LEVEL_WIDTH))
    enemy_heatmap = np.zeros((LEVEL_HEIGHT, LEVEL_WIDTH))

    for lvl in levels:
        arr = lvl[0].cpu().numpy()
        for tid in GROUND_TILES:
            ground_heatmap += (arr == tid)
        for tid in ENEMIES:
            enemy_heatmap += (arr == tid)
        combined_heatmap += (arr != AIR_ID)

    n = len(levels)
    ground_heatmap /= n
    enemy_heatmap /= n
    combined_heatmap /= n

    plt.figure(figsize=(15, 12))

    plt.subplot(3, 1, 1)
    sns.heatmap(combined_heatmap, cmap="YlOrRd", vmin=0, vmax=1)
    plt.title('Heatmap of All Non-Empty Tiles')

    plt.subplot(3, 1, 2)
    sns.heatmap(ground_heatmap, cmap="YlGn", vmin=0, vmax=1)
    plt.title('Heatmap of Ground Tiles')

    plt.subplot(3, 1, 3)
    if np.max(enemy_heatmap) > 0:
        sns.heatmap(enemy_heatmap, cmap="Reds", vmin=0, vmax=0.5)
    else:
        plt.text(0.5, 0.5, 'No enemy tiles detected', ha='center', va='center',
                 transform=plt.gca().transAxes, fontsize=14)
    plt.title('Heatmap of Enemy Tiles')

    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, 'tile_heatmaps.png'), dpi=300)
    plt.close()

# Run all visualizations and print progress
print("Generating visualizations for paper...")
plot_loss_curves()
print("- Loss curves plotted")

print("- Generating sample levels for analysis...")
sample_levels = generate_sample_levels(num_samples=30, seed=42)
print(f"- Generated {len(sample_levels)} sample levels")

level_metrics = calculate_metrics(sample_levels)
print("- Calculated level metrics")

plot_expressive_range(level_metrics)
print("- Expressive range plotted")

plot_tile_distribution(sample_levels)
print("- Tile distribution plotted")

plot_tile_heatmaps(sample_levels)
print("- Tile heatmaps plotted")

print(f"All visualizations saved to {FIGURES_DIR}")

# Print summary stats
print("\nQuantitative Results Summary:")
print(f"- Linearity: mean={np.mean([m['linearity'] for m in level_metrics]):.3f}, std={np.std([m['linearity'] for m in level_metrics]):.3f}")
print(f"- Leniency: mean={np.mean([m['leniency'] for m in level_metrics]):.3f}, std={np.std([m['leniency'] for m in level_metrics]):.3f}")
print(f"- Tile Entropy: mean={np.mean([m['tile_entropy'] for m in level_metrics]):.3f}, std={np.std([m['tile_entropy'] for m in level_metrics]):.3f}")


Generating visualizations for paper...
- Loss curves plotted
- Generating sample levels for analysis...
- Generated 30 sample levels
- Calculated level metrics
- Expressive range plotted
- Tile distribution plotted
- Tile heatmaps plotted
All visualizations saved to ./generated_levels/figures

Quantitative Results Summary:
- Linearity: mean=1.030, std=0.462
- Leniency: mean=0.130, std=0.089
- Tile Entropy: mean=0.863, std=0.066


In [36]:
import os, random

def generate_multiple_levels(output_folder, easy_count=5, medium_count=5, hard_count=5, seed=None):
    """
    Generate multiple levels across easy, medium, and hard presets, saving to disk.
    """
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    # Set random seed if provided
    if seed is not None:
        random.seed(seed)

    counts = {"easy": 0, "medium": 0, "hard": 0}  # Count levels by difficulty

    # Generate easy levels
    print(f"\nGenerating {easy_count} easy levels...")
    for i in range(easy_count):
        level_seed = random.randint(1, 10000) if seed else 0
        fname = f"level_easy_{i:04d}.txt"
        full_path = os.path.join(output_folder, fname)

        # Copy preset and add seed
        cfg = PRESETS["easy"]
        if level_seed:
            cfg = DifficultyConfig(**{**cfg.__dict__, "seed": level_seed})

        parts = []
        for _ in range(math.ceil(LEVEL_WIDTH / PATCH)):
            with torch.no_grad():
                z = torch.randn(1, LATENT_DIM, device=device)
                parts.append(clean_chunk(to_int(Gnet(z).cpu()).squeeze(0)))

        lvl = stitch(parts)
        lvl = carve_random_pits(lvl, cfg)
        lvl = prune_sky(lvl, cfg)

        # Place Mario and goal
        lvl[0, :, 1] = AIR_ID  # Clear space
        lvl[0, LEVEL_HEIGHT - 8, 1] = START
        lvl[0, LEVEL_HEIGHT - 2, LEVEL_WIDTH - 2] = GOAL

        # Add some coins
        if COIN_ID != AIR_ID:
            for y in range(4, LEVEL_HEIGHT - 4):
                for x in range(4, LEVEL_WIDTH - 4):
                    if lvl[0, y, x] == AIR_ID and random.random() < 0.03:
                        lvl[0, y, x] = COIN_ID

        with open(full_path, "w") as f:
            f.write("\n".join(ascii_grid(lvl)))

        print(f"  Saved {fname} to {output_folder}")
        counts["easy"] += 1

    # Generate medium levels
    print(f"\nGenerating {medium_count} medium levels...")
    for i in range(medium_count):
        level_seed = random.randint(1, 10000) if seed else 0
        fname = f"level_medium_{i:04d}.txt"
        full_path = os.path.join(output_folder, fname)

        cfg = PRESETS["medium"]
        if level_seed:
            cfg = DifficultyConfig(**{**cfg.__dict__, "seed": level_seed})

        parts = []
        for _ in range(math.ceil(LEVEL_WIDTH / PATCH)):
            with torch.no_grad():
                z = torch.randn(1, LATENT_DIM, device=device)
                parts.append(clean_chunk(to_int(Gnet(z).cpu()).squeeze(0)))

        lvl = stitch(parts)
        lvl = carve_random_pits(lvl, cfg)
        lvl = prune_sky(lvl, cfg)

        lvl[0, :, 1] = AIR_ID
        lvl[0, LEVEL_HEIGHT - 8, 1] = START
        lvl[0, LEVEL_HEIGHT - 2, LEVEL_WIDTH - 2] = GOAL

        if COIN_ID != AIR_ID:
            for y in range(4, LEVEL_HEIGHT - 4):
                for x in range(4, LEVEL_WIDTH - 4):
                    if lvl[0, y, x] == AIR_ID and random.random() < 0.015:
                        lvl[0, y, x] = COIN_ID

        with open(full_path, "w") as f:
            f.write("\n".join(ascii_grid(lvl)))

        print(f"  Saved {fname} to {output_folder}")
        counts["medium"] += 1

    # Generate hard levels
    print(f"\nGenerating {hard_count} hard levels...")
    for i in range(hard_count):
        level_seed = random.randint(1, 10000) if seed else 0
        fname = f"level_hard_{i:04d}.txt"
        full_path = os.path.join(output_folder, fname)

        cfg = PRESETS["hard"]
        if level_seed:
            cfg = DifficultyConfig(**{**cfg.__dict__, "seed": level_seed})

        parts = []
        for _ in range(math.ceil(LEVEL_WIDTH / PATCH)):
            with torch.no_grad():
                z = torch.randn(1, LATENT_DIM, device=device)
                parts.append(clean_chunk(to_int(Gnet(z).cpu()).squeeze(0)))

        lvl = stitch(parts)
        lvl = carve_random_pits(lvl, cfg)
        lvl = prune_sky(lvl, cfg)

        lvl[0, :, 1] = AIR_ID
        lvl[0, LEVEL_HEIGHT - 8, 1] = START
        lvl[0, LEVEL_HEIGHT - 2, LEVEL_WIDTH - 2] = GOAL

        if COIN_ID != AIR_ID:
            for y in range(4, LEVEL_HEIGHT - 4):
                for x in range(4, LEVEL_WIDTH - 4):
                    if lvl[0, y, x] == AIR_ID and random.random() < 0.005:
                        lvl[0, y, x] = COIN_ID

        with open(full_path, "w") as f:
            f.write("\n".join(ascii_grid(lvl)))

        print(f"  Saved {fname} to {output_folder}")
        counts["hard"] += 1

    # Print summary
    total = sum(counts.values())
    print(f"\nGeneration complete! Created {total} levels:")
    for k in ["easy", "medium", "hard"]:
        print(f"  {k.capitalize()}: {counts[k]}")
    print(f"All levels saved to: {output_folder}")

# Set output directory and call the generator
MY_OUTPUT_FOLDER = "./test_demo"

generate_multiple_levels(
    output_folder=MY_OUTPUT_FOLDER,
    easy_count=3,
    medium_count=3,
    hard_count=3,
    seed=None  # Use fixed value for reproducibility
)


Generating 3 easy levels...
  Saved level_easy_0000.txt to ./test_demo
  Saved level_easy_0001.txt to ./test_demo
  Saved level_easy_0002.txt to ./test_demo

Generating 3 medium levels...
  Saved level_medium_0000.txt to ./test_demo
  Saved level_medium_0001.txt to ./test_demo
  Saved level_medium_0002.txt to ./test_demo

Generating 3 hard levels...
  Saved level_hard_0000.txt to ./test_demo
  Saved level_hard_0001.txt to ./test_demo
  Saved level_hard_0002.txt to ./test_demo

Generation complete! Created 9 levels:
  Easy: 3
  Medium: 3
  Hard: 3
All levels saved to: ./test_demo
